<a href="https://colab.research.google.com/github/AkashaMaqsood134/python1/blob/master/TicTacToe(server_client).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Tic tac toe client

import socket

def intro():
    """Rules of the game."""
    print("Hello! Welcome to Pam's Tic Tac Toe game!\n")
    print("Rules: Player 1 and player 2, represented by X and O, take turns "
          "marking the spaces in a 3x3 grid. The player who succeeds in placing "
          "three of their marks in a horizontal, vertical, or diagonal row wins.\n")
    input("Press enter to continue...\n")
    print("\n")

def main():
    intro()
    sc = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    sc.connect(("127.0.0.1",65432))

    awaiting_move = False

    while True:
        try:
            msg = sc.recv(1024).decode()
            if not msg:
                break
            print(msg)
            if "Current Board:" in msg:
                awaiting_move = True

            if awaiting_move:
                 awaiting_move = False
                 move = input()
                 sc.send(move.encode())


            if any(end_phrase in msg for end_phrase in ["wins", "draw", "tie"]):
                print("Game over!")
                break

        except Exception as e:
            print(f"\nConnection issue: {e}")
            break
    sc.close()

if __name__ == "__main__":
    main()

In [ ]:

#Tic tac toe Server
import socket
import threading

def printPretty():
    """Returns the board as a formatted string"""
    board_str = "\n---+---+---\n"
    for row in board:
        board_str += " | ".join(row) + "\n---+---+---\n"
    return board_str

def isWinner():
    """Checks if a player has won the game"""
    for i in range(3):
        if board[i][0] == board[i][1] == board[i][2] != " ":
            return True
        if board[0][i] == board[1][i] == board[2][i] != " ":
            return True
    if board[0][0] == board[1][1] == board[2][2] != " ":
        return True
    if board[0][2] == board[1][1] == board[2][0] != " ":
        return True
    return False


def isFull():
    """Checks if the board is full"""
    return all(cell != " " for row in board for cell in row)
def isDraw():
    """Checks if the game is a draw"""
    return isFull() and not isWinner()

#for game updates
def broadcast(msg):
    """Sends a message to all players"""
    for player in list(players.values()):
        try:
            player.send(msg.encode())
        except:
            pass

def startGaming(cs, player_id):
    global turn, game_over
    cs.send("Waiting for another player to join...\n".encode())
    while len(players) < 2:
        threading.Event().wait(0.1) #pause execution

    cs.send(f"\nYou are Player {player_id + 1} ({symbols[player_id]}). Game starts now!\n".encode())

    while not game_over:
        with game_lock: #single player
            if turn == player_id:
                try:
                    cs.send(("\nCurrent Board:\n" + printPretty() +
                              "\nYour turn! - Enter row and column (e.g., '0 1'): ").encode())

                    cs.settimeout(30)
                    move = cs.recv(1024).decode().strip()
                    row, col = map(int, move.split())

                    if 0 <= row < 3 and 0 <= col < 3 and board[row][col] == " ":
                        board[row][col] = symbols[player_id]
                        broadcast("\nUpdated Board:\n" + printPretty())

                        if isWinner():
                            game_over = True
                            broadcast(f"\nPlayer {player_id + 1} ({symbols[player_id]}) wins!\n")
                            break
                        elif isDraw():
                            game_over = True
                            broadcast("\nIt's a draw!\n")
                            break
                        else:
                            turn = 1 - turn
                    else:
                        cs.send("Invalid move! Try again.\n".encode())
                except (ValueError, socket.timeout):
                    cs.send("Invalid input or timeout. Enter row and column as '0 1'.\n".encode())
                except Exception as e:
                    print(f"Error: {e}")
                    break

    cs.close()
    del players[player_id]

    if len(players) == 0:
        game_over = True

def main():
    global players, turn, symbols, game_over, board, game_lock

    board = [[" " for _ in range(3)] for _ in range(3)]
    players = {}
    turn = 0
    symbols = ["X", "O"]
    game_over = False
    game_lock = threading.Lock()

    server = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    server.bind(("127.0.0.1", 65432))
    server.listen(2)

    print("Server started. Waiting for players to connect...")

    player_id = 0
    while len(players) < 2:
        c_sock, addr = server.accept()
        print(f"Player {player_id + 1} connected from {addr}")
        players[player_id] = c_sock
        threading.Thread(target=startGaming, args=(c_sock, player_id)).start()
        player_id += 1

    while not game_over:
        threading.Event().wait(1)

    print("Game ended")
    server.close()

if __name__ == "__main__":
    main()
